<h1>Data Retrieval</h1>
<p>
the dataset are manually encoded questionnaire from various math resources such as DepEd produced modules posted online, Grade 10 Math Books, mathematics dataset
</p>
<h5>Resources:</h5>
<ul>
    <li>
        DepEd Modules:
        <ul>
            <li>https://www.depedk12.com/2021/08/grade-10-self-learning-modules-quarter-1.html</li>
            <li>https://www.scribd.com/document/502974315/Math-10-Q1-Mod10</li>
            <li>https://www.scribd.com/document/480952622/Math10-Q1-Mod14-Illustrating-polynomial-equations-Lessons-1-2-v3</li>
        </ul>
    </li>
    <li>Escaner J.M.L., Catalla D.L., & Granada I.O. (n.d.) Spiral Math 10 2nd Edition</li>
    <li>Melosantos L.A.B, Antonio J.F., Robles S. J., Sacluti J.R.,& Bruce R. M.(2015) Math Connections in the Digital Age 10. Sibs Publishing House Inc.</li>
    <li>Matheamtics Dataset: https://github.com/deepmind/mathematics_dataset</li>
</ul>


<h3>CONVERT CSV FILE TO PANDAS DATAFRAME</h3>

In [1]:
import pandas as pd

# df1 = dataframe of manually encoded datasets.

df1 = pd.read_csv('dataset-december.csv', encoding='utf-8')
df1 = df1.drop(columns = ['id'])
df1

,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence
...,...,...
840,Determine the zeros of the polynomial function...,polynomial_equation
841,Determine the zeros of the polynomial function...,polynomial_equation
842,Determine the zeros of the polynomial function...,polynomial_equation
843,Determine the zeros of the polynomial function...,polynomial_equation


In [2]:
# display instances
df1.label.value_counts()

series_and_sequence    318
polynomials            318
polynomial_equation    209
Name: label, dtype: int64

Adding polynomial_equation data to supply 109 instances of question from mathematics dataset.  

In [3]:
#df2 additional dataset for polynomial equations

df2 = pd.DataFrame(columns=['question', 'label'])
with open("algebra__polynomial_roots.txt", "r") as data:
    #First 109 instance of Dataset      
    list_item = data.readlines()[:217]
    for item in range(0,len(list_item),2):
        df2 = df2.append({"question":list_item[item], "label":"polynomial_equation"}, ignore_index=True)
    data.close()

initial_dataset = df1.append(df2, ignore_index=True)
print('DATASET PREVIEW')
print(initial_dataset)
# initial_dataset.to_csv("initial_dataset.csv", encoding ='utf-8') 

DATASET PREVIEW
                                              question                label
0            It is the sum of the terms of a sequence.  series_and_sequence
1    Find the sum of the first ten terms of the ari...  series_and_sequence
2    Find the sum of the first 25 terms of the arit...  series_and_sequence
3    The sum of the first 10 terms of an arithmetic...  series_and_sequence
4    The third term of an arithmetic sequence is −1...  series_and_sequence
..                                                 ...                  ...
949                Factor -35*k**2 + 27415*k - 7830.\n  polynomial_equation
950       Factor -6*n**4/5 - 116*n**3/5 - 38*n**2/5.\n  polynomial_equation
951  What is w in 3*w**3 - 3561*w**2 + 1058505*w - ...  polynomial_equation
952    Find v, given that 12*v**2 - 585*v - 147 = 0.\n  polynomial_equation
953  Determine z so that 3*z**3 + 234*z**2/5 + 864*...  polynomial_equation

[954 rows x 2 columns]


re-sampled the data using mathematics dataset 

In [4]:
print('DATASET INFORMATION')
print(initial_dataset.label.value_counts())

DATASET INFORMATION
series_and_sequence    318
polynomials            318
polynomial_equation    318
Name: label, dtype: int64


<h1>Data Cleaning</h1>

To generalize a formula, and expressions, equation, turn them into one single format
example: 

Math Expression:

$$a_n+\frac{\left(-1\right)^n}{n^2}$$ 


In LaTex (How it is encoded on the Dataset)
<br>
<br>
<br>
<center>\$a_n+\frac{\left(-1\right)^n}{n^2}\$</center>
<br>
<br>
<br>
In Freeform formula (Converted via library Sympy): (It rearrange the solution into PEMDAS format)
<br>
<br>
<br>
<center>(-1)**n/(n**2) + a_n</center>

Sympy Library converts freeform into expression tree which arranges the expression on how it will be processed if solved. so similarly built expressions can be counted as one token.

In [5]:
import re

def change_to_dollar(exp):
    exp = re.sub("\.\.\.|\\dots|…","x",exp)
    exp = re.sub("=","$ $",exp)
    return exp

for i in range(len(initial_dataset)):
    qs = initial_dataset.loc[i, 'question']
    fx = re.search('[$].+?[=].+?[$]',qs)
    len_check = len(re.findall('[$].+?[=].+?[$]', qs))
    iplus = 0
    while iplus<len_check:
        iplus += 1
        qs = qs[:fx.span()[0]] + " " + change_to_dollar(fx.group()) + " " + qs[fx.span()[1]:]
        #check for the next LaTex Enclosure if any         
        fx = re.search('[$].+?[=].+?[$]', qs)
    #the new question is added to index i       
    initial_dataset.loc[i,'question'] = qs
    

# Since Sympy cannot read sqrt latex we'll convert it into exponent format using regex
pattern = r"\\sqrt\{(.+?)\}"
replacement = r"(\1)^{\\frac{1}{2}}"

dataset_sympytolatex = initial_dataset.replace(pattern, replacement, regex=True)

print("dataset preview ready for converting LaTeX expression enclosed in $$ into freeform expression")
dataset_sympytolatex.head()

dataset preview ready for converting LaTeX expression enclosed in $$ into freeform expression


,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence


<h3>CONVERT EACH LATEX FORMULA INTO FREEFORM EQUATION USING LATEX2SYMPY LIBRARY</h3>

In [6]:
from latex2sympy2 import latex2sympy
import re

# copy pasted from previous code so I just reassigned the old used df so I have minimal changes to the dataset
df = dataset_sympytolatex

# Creates a new dataframe for updated question with freeform formula
new_df = pd.DataFrame(columns=['question', 'label'] )

# looping through all data    
for i in range(len(df)):
    new_df.loc[i,'label'] = df.loc[i,'label']
    test = df.loc[i, 'question']
    #regex value for searching all characters from '$' to the next '$' symbol
    check = re.search('[$].+?[$]',test)
    #checks how many instance of latex enclosure are there in the question     
    check_len = len(re.findall('[$].+?[$]',test))
    iteration = 0
    while iteration<check_len:
        iteration += 1
        # concatenate the question string from all characters before formula + formula + all characters after formula
        # formula = re.sub(" ","",str(latex2sympy(r"{}".format(check.group()))))
        # the formula is a converted LaTex to Sympy then removed all white spaces
        # If new dataset is introduced you can print the index of question and the group expression that it returned error.
        # print("change problem in index", i)
        # print(check.group())
        test = test[:check.span()[0]] + " " + re.sub(" ","",str(latex2sympy(r"{}".format(check.group())))) + " " + test[check.span()[1]:]
        #check for the next LaTex Enclosure if any         
        check = re.search('[$].+?[$]',test)
    #the new question is added to index i       
    new_df.loc[i,'question'] = test

print("Preview of converted ")
new_df.head()

Preview of converted 


,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence


<h3>Restructuring Formulas and Removing Unnecessary Characters</h3>

In [7]:
# removing the spaces between from converted freeform expression
new_df = new_df.replace(' \+ ', '+', regex=True)
new_df = new_df.replace(' - ', '-', regex=True)
# We remove the equal sign so that the sympy library would not automatically evaluate the expression
new_df = new_df.replace(' = ', ' ', regex=True)
# cleaning the unnecessary characters
new_df = new_df.replace('\n','', regex=True) #white space
new_df = new_df.replace('\?','', regex=True) #Question marks
new_df = new_df.replace('\'','', regex=True) #quotation marks
new_df = new_df.replace(',','', regex=True)  #comma
new_df = new_df.replace('[\.]','', regex=True) #periods
new_df = new_df.replace('\t', ' ', regex=True) #tabs
new_df = new_df.replace('[;]', ' ', regex=True) #tabs
new_df = new_df.replace(':', ' ', regex=True) #tabs
new_df = new_df.replace(r"\_\_+", "__", regex=True)

# you can call or convert the dataset_freeform dataframe to view the cleaned dataset before tokenizing into expression trees
dataset_freeform = new_df

<h3>CREATE A LIST OF QUESTIONS TO BE USED FOR BAG OF WORDS</h3>

In [8]:
question_list = new_df.drop(columns=['label']).values.tolist()
question_list[:10]

[['It is the sum of the terms of a sequence'],
 ['Find the sum of the first ten terms of the arithmetic sequence 41016'],
 ['Find the sum of the first 25 terms of the arithmetic sequence 17 22 2732 …'],
 ['The sum of the first 10 terms of an arithmetic sequence is 530 What is the first term if the last term is 80 and the common difference is 2'],
 ['The third term of an arithmetic sequence is −12 and the seventh term is 8 What is the sum of the first 10 terms'],
 ['Find the sum of the first 50 terms of the arithmetic sequence if the first term is 21 and the twentieth term is 154'],
 ['Find the sum of the first eighteen terms of the arithmetic sequence whose nth term is '],
 ['The first term of an arithmetic sequence is 5 the last term is 45 and the sum is 275 Find the number of terms'],
 ['If the first n terms of the arithmetic sequence 20 18 16 are added how many of these terms will be added to get a sum of -100'],
 ['A yaya receives a starting annual salary of Php 3600000 with a year

<h3>GENERALIZING EXPRESSIONS SO THAT IT WOULD NOT BE TREATED AS DFFERENT TOKENS</h3>
<br>
if $x^3+4x^2+5x+1$ has a similar structure to $y^3+5y^2+2y+3$ then they must be counted as one token
 
<table>
    <tr>
        <td>exp</td>
        <td>LaTex</td>
        <td>Sympy</td>
        <td>Expression Tree</td>
    </tr>
    <tr>
        <td>1</td>
        <td>x^3+4x^2+5x+1</td>
        <td>x**3 + 4*x**2 + 5*x + 1</td>
        <td>Add(Pow(Symbol('x'), Integer(3)), Mul(Integer(4), Pow(Symbol('x'), Integer(2))), Mul(Integer(5), Symbol('x')), Integer(1))</td>
    </tr>
    <tr>
        <td>2</td>
        <td>y^3+5y^2+2x+3</td>
        <td>x**3 + 4*x**2 + 2*y + 1</td>
        <td>Add(Pow(Symbol('y'), Integer(3)), Mul(Integer(5), Pow(Symbol('y'), Integer(2))), Mul(Integer(2), Symbol('y')), Integer(3))</td>
    </tr>
</table>

We'll turn every Sympy formula into Expression Trees so that it formats the Sympy form into more structured and in order manner that increases its likeness to other similar structure expressions. we'll apply the following code to generalize each number and variable into one similar structure

            text = re.sub("\d+","(d)",text)
            text = re.sub("[(]['].+?['][)]","(x)",text)


<table>
    <tr>
        <td>exp</td>
        <td>Expression Tree</td>
        <td>After Regex</td>
    </tr>
    <tr>
        <td>1</td>
        <td>Add(Pow(Symbol('x'), Integer(3)), Mul(Integer(4), Pow(Symbol('x'), Integer(2))), Mul(Integer(5), Symbol('x')), Integer(1))</td>
        <td>'Add(Pow(Symbol(x), Integer((d))), Mul(Integer((d)), Pow(Symbol(x), Integer((d)))), Mul(Integer((d)), Symbol(x)), Integer((d)))'</td>
    </tr>
    <tr>
        <td>2</td>
        <td>Add(Pow(Symbol('y'), Integer(3)), Mul(Integer(5), Pow(Symbol('y'), Integer(2))), Mul(Integer(2), Symbol('y')), Integer(3))</td>
        <td>'Add(Pow(Symbol(x), Integer((d))), Mul(Integer((d)), Pow(Symbol(x), Integer((d)))), Mul(Integer((d)), Symbol(x)), Integer((d)))'</td>
    </tr>
</table>


The two expressions are now syntactically similar. and can be represented as one token

In [9]:
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
import re

def tokenize_formula(expression):
    try:
        return srepr(parse_expr(expression, evaluate=False))
    except:
        return expression

for i in range(len(question_list)):
    item_question = question_list[i][0]
    tokenize_question_item = item_question.split()
    for x in range(len(tokenize_question_item)):
        check = re.findall('[\+]|[\*]|[-]|[/]|[0-9]',tokenize_question_item[x])
        if len(tokenize_question_item[x]) == 1:
            check = True 
        if check: 
            # Uncomment print functions to display the conversion of each math tokens and 
            # their index into expression tree form
            #print("index: ",i)
            #print(tokenize_question_item[x])
            text = re.sub("\s|[,]","",str(tokenize_formula(tokenize_question_item[x])))
            #print(text)
            text = re.sub("\d+","(d)",text)
            text = re.sub("[(]['].+?['][)]","(x)",text)
            text = re.sub('\s+', '', text)
            tokenize_question_item[x] = text

    question_list[i] = ' '.join(tokenize_question_item)

In [10]:
# Generating the dataset
labels = new_df.loc[:,'label'].values.tolist()
# print(labels)
# print(question_list)
# generated_dataset = pd.DataFrame([question_list, labels], columns=['question', 'label'])
# generated_dataset.to_csv('dataset_after_transformation.csv', encoding='utf-8')
lab_ques = {'question':list(question_list), 'label':list(labels)}
generated_dataset = pd.DataFrame(lab_ques)
# generated_dataset.to_csv('dataset_after_transformation.csv', encoding='utf-8')
generated_dataset

,question,label
0,It is the sum of the terms of Symbol(x) sequence,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first Integer((d)) terms o...,series_and_sequence
3,The sum of the first Integer((d)) terms of an ...,series_and_sequence
4,The third term of an arithmetic sequence is −(...,series_and_sequence
...,...,...
949,Factor Add(Mul(Integer(-(d))Integer((d))Pow(Sy...,polynomial_equation
950,Factor Add(Mul(Integer(-(d))Integer((d))Pow(In...,polynomial_equation
951,What is Symbol(x) in Add(Mul(Integer((d))Pow(S...,polynomial_equation
952,Find Symbol(x) given that Add(Mul(Integer((d))...,polynomial_equation


<h3>APPLYING THE VECTORIZER</h3>

In [11]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

words = stopwords.words('english')

vect = CountVectorizer(token_pattern = '[a-zA-Z0-9,=+-<>()_{}]+',stop_words = words, min_df=0)
vect.fit(question_list)

CountVectorizer(min_df=0,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                token_pattern='[a-zA-Z0-9,=+-<>()_{}]+')

In [12]:
print("Column Names and their id")
keys = list(vect.vocabulary_.keys())
values = list(vect.vocabulary_.values())

with pd.option_context('display.max_rows', None,):
    print((pd.DataFrame(values ,index = keys, columns = ['id'])).sort_values(by = ['id']))

Column Names and their id
                                                     id
(binomial)                                            0
(d)                                                   1
(d)+(d)+(d)+                                          2
(d)cm                                                 3
(d)m                                                  4
(d)n                                                  5
(d)nd                                                 6
(d)rd                                                 7
(d)s_(d)                                              8
(d)st                                                 9
(d)th                                                10
(h)                                                  11
(if                                                  12
(l)                                                  13
(polynomial)                                         14
(w)                                                  15
-(d)                  

In [13]:
print("Total Columns")
print(len(vect.get_feature_names_out()))

Total Columns
894


<h3>TRANSFORMING QUESTION LIST TO BAG-OF-WORDS</h3>

In [14]:
bag_of_words = vect.transform(question_list)
format(bag_of_words.toarray())

'[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]]'

In [15]:
import scipy.sparse

bow_Matrix = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns = vect.get_feature_names_out())
X = bow_Matrix.values
y = new_df.loc[:,"label"].values
bow_Matrix.head()

,(binomial),(d),(d)+(d)+(d)+,(d)cm,(d)m,(d)n,(d)nd,(d)rd,(d)s_(d),(d)st,...,worth,would,write,x,yaya,year,yearly,years,zero,zeros
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>APPLYING THE MACHINE LEARNING MODEL</h3>

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state= 42)

In [17]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100,random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[79  2  0]
 [ 6 78  0]
 [ 1  0 73]]
                     precision    recall  f1-score   support

polynomial_equation       0.92      0.98      0.95        81
        polynomials       0.97      0.93      0.95        84
series_and_sequence       1.00      0.99      0.99        74

           accuracy                           0.96       239
          macro avg       0.96      0.96      0.96       239
       weighted avg       0.96      0.96      0.96       239



In [19]:
accuracy_score(y_test, y_pred)

0.9623430962343096

Importing the Model and Vectorizer using Pickle

In [20]:
# import pickle
## Uncomment to produce the file
## save the model to disk
# model = 'rfa_model_v1.sav'
# pickle.dump(classifier, open(model, 'wb'))
# myvectorizer = 'vectorizer_v1.sav'
# pickle.dump(vect, open(myvectorizer, 'wb'))

In [21]:
# Using Bag Of Words for Importing to other ML TOOLS (Weka)
# create_csv = bow_Matrix
# create_csv= create_csv.assign(ML_class=y)
# print(create_csv)
# CREATING A DATASET FOR EDA PURPOSES
# create_csv.to_csv("BOW_dataset_v1.csv", encoding='utf-8') 